In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { min-width:1500px !important; }</style>"))

import os
import sys
sys.path.append('..')

import time
import datetime

import importlib

import tensorflow as tf
from tensorflow import keras

import src
from src.data import cifar100
from src.models import basic, factory
from src.meta.meta import Metadata
from src.utils import debug_utils, file_utils, io_utils

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S   '), tf.__name__, '-', tf.__version__, sep='')

/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

2019-08-14 12:27:14   tensorflow-1.14.0


/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [2]:
from src.utils.logger import Logging
logger = Logging.get_logger(__name__)
Logging.attach_stdout()

# Setup

In [3]:
_NAME = basic.NAME
_EPOCH = 10

filepath_md = file_utils.model_filepath_md(_NAME, _EPOCH)
print('filepath_md:            ', filepath_md)
filepath_pb = file_utils.model_filepath_pb(_NAME, _EPOCH)
print('filepath_pb:            ', filepath_pb)
filepath_weights_h5 = file_utils.model_filepath_weights_h5(_NAME, _EPOCH)
print('filepath_weights_h5:    ', filepath_weights_h5)
filepath_tflite = file_utils.model_filepath_tflite(_NAME, _EPOCH)
print('filepath_tflite:        ', filepath_tflite)
filepath_edgetpu_tflite = file_utils.model_filepath_edgetpu_tflite(_NAME, _EPOCH)
print('filepath_edgetpu_tflite:', filepath_edgetpu_tflite)

output_data_file = 'tmp/test_data_%s.csv' % datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('output_data_file:       ', output_data_file)

TRIALS = 10
TEST_SIZES = [1, 10, 100, 1000]
REPEAT = 2

filepath_md:             /space/code/deep-learning/models/basic/010/basic_010.md
filepath_pb:             /space/code/deep-learning/models/basic/010/basic_010.pb
filepath_weights_h5:     /space/code/deep-learning/models/basic/010/basic_010_weights.h5
filepath_tflite:         /space/code/deep-learning/models/basic/010/basic_010_int8.tflite
filepath_edgetpu_tflite: /space/code/deep-learning/models/basic/010/basic_010_int8_edgetpu.tflite
output_data_file:        tmp/test_data_2019-08-14-12:27:14.csv


In [4]:
%%bash

find ../models | grep basic | grep -E '(010|009)' | sort

../models/basic/009
../models/basic/009/basic_009_checkpoint.h5
../models/basic/009/basic_009.h5
../models/basic/009/basic_009.md
../models/basic/009/basic_009_weights.h5
../models/basic/010
../models/basic/010/basic_010_checkpoint.h5
../models/basic/010/basic_010.h5
../models/basic/010/basic_010_int8_edgetpu.log
../models/basic/010/basic_010_int8_edgetpu.tflite
../models/basic/010/basic_010_int8.tflite
../models/basic/010/basic_010.md
../models/basic/010/basic_010.pb
../models/basic/010/basic_010_weights.h5


# Data

In [5]:
from src.data import cifar100

In [6]:
(TRAIN_X, TRAIN_Y), (TEST_X, TEST_Y) = cifar100.load_data(normalize=True)

# Keras

In [7]:
from src.keras.model import KerasModel
importlib.reload(src.keras.model)
from src.keras.model import KerasModel

In [8]:
f_construct_keras_model = factory.get_model_create_func(_NAME, cifar100.INPUT_SHAPE, len(cifar100.CLASSES))

USE = 'create'

if USE == 'huli':
    ret, keras_model = KerasModel.from_h5(filepath_md, '../resnet50_csv_16.h5')

elif USE == 'create':
    ret, keras_model = KerasModel.from_factory_func(_NAME, f_construct_keras_model)
    assert ret == 0
    keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    keras_model.fit(TRAIN_X, TRAIN_Y, epochs=1, initial_epoch=0)

elif USE == 'pretrained':
    ret, keras_model = KerasModel.from_weights_h5(filepath_md, f_construct_keras_model, filepath_weights_h5)   

assert ret == 0

W0814 12:27:14.849013 139970022432896 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2019-08-14 12:27:14,865 DEBUG [src.keras.model:39] (MainThread) [basic|0|keras] Setting metadata input names to ['basic0:0']
2019-08-14 12:27:14,865 DEBUG [src.keras.model:47] (MainThread) [basic|0|keras] Setting metadata out names to ['basic2/Softmax:0']
2019-08-14 12:27:14,896 DEBUG [src.keras.model:203] (MainThread) [basic|0|keras] keras tensorboard callback attached. Visualize by running: 
2019-08-14 12:27:14,897 DEBUG [src.keras.model:204] (MainThread) [basic|0|keras] > tensorboard --logdir=/space/code/deep-learning/tmp/logs/tensorboard/basic/20190814-122714
2019-08-14 12:27:14,897 DEBUG [src.keras.model:206] (MainThread) [basic|0|keras] keras checkpoint callback attached. Logging to /space/code/deep-learning/models/basic/{epoch:03d}/basic_{epoch:03d}_checkpoint.h5...
2019-08-14 12:27:14,898 INFO  [src.keras.model:213] (MainThread) [basic|0|keras] Creating /space/code/deep-learning/models/basic/001 (if not already exists)...
50000/50000 [==============================] - 3s 65us/s

In [9]:
%%bash

find ../models/basic/001 -type f | sort | xargs ls -l

-rw-rw-r-- 1 maka maka 3706776 Aug 14 12:27 ../models/basic/001/basic_001_checkpoint.h5
-rw-rw-r-- 1 maka maka 3707824 Aug 13 15:36 ../models/basic/001/basic_001.h5
-rw-rw-r-- 1 maka maka     189 Aug 13 15:36 ../models/basic/001/basic_001.md
-rw-rw-r-- 1 maka maka 1240912 Aug 13 15:36 ../models/basic/001/basic_001_weights.h5


# SavedGraph

In [10]:
saved_model_path = "./saved_models/"+str(int(time.time()))
tf.contrib.saved_model.save_keras_model(keras_model.keras_model, saved_model_path)

W0814 12:27:19.086565 139970022432896 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0814 12:27:19.131745 139970022432896 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 12:27:19.132856 139970022432896 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/t

In [11]:
%%bash

find saved_models -type f | sort | xargs ls -l

-rw-rw-r-- 1 maka maka    1122 Aug 14 11:49 saved_models/1565808599/assets/saved_model.json
-rw-rw-r-- 1 maka maka   12813 Aug 14 11:49 saved_models/1565808599/saved_model.pb
-rw-rw-r-- 1 maka maka      75 Aug 14 11:49 saved_models/1565808599/variables/checkpoint
-rw-rw-r-- 1 maka maka 1229200 Aug 14 11:49 saved_models/1565808599/variables/variables.data-00000-of-00002
-rw-rw-r-- 1 maka maka     279 Aug 14 11:49 saved_models/1565808599/variables/variables.data-00001-of-00002
-rw-rw-r-- 1 maka maka     281 Aug 14 11:49 saved_models/1565808599/variables/variables.index
-rw-rw-r-- 1 maka maka    1122 Aug 14 12:00 saved_models/1565809209/assets/saved_model.json
-rw-rw-r-- 1 maka maka   87366 Aug 14 12:00 saved_models/1565809209/saved_model.pb
-rw-rw-r-- 1 maka maka      75 Aug 14 12:00 saved_models/1565809209/variables/checkpoint
-rw-rw-r-- 1 maka maka 3687624 Aug 14 12:00 saved_models/1565809209/variables/variables.data-00000-of-00002
-rw-rw-r-- 1 maka maka    1285 Aug 14 12:00 saved_mode

# TensorRT Convert

In [12]:
%%bash

rm -rf trt_models/*

In [13]:
input_saved_model_dir = saved_model_path
output_saved_model_dir = 'trt_models'

In [14]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

converter = trt.TrtGraphConverter(input_saved_model_dir=input_saved_model_dir)
converter.convert()
converter.save(output_saved_model_dir)

W0814 12:27:19.770522 139970022432896 deprecation.py:323] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/compiler/tensorrt/trt_convert.py:245: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0814 12:27:19.784785 139970022432896 deprecation.py:323] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0814 12:27:19.797768 139970022432896 depr

In [15]:
%%bash

find trt_models | sort

trt_models
trt_models/saved_model.pb
trt_models/variables


In [16]:
with tf.Session() as sess:
    # First load the SavedModel into the session    
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], output_saved_model_dir)
    #output = sess.run([output_tensor], feed_dict={input_tensor: input_data})

ValueError: The passed save_path is not a valid checkpoint: trt_models/variables/variables

In [ ]:
%%bash

find trt_models | sort | xargs ls -l

# Debug

In [ ]:
keras.backend.clear_session()
del keras_model.keras_model
del keras_model
keras.backend.clear_session()

In [ ]:
help(freeze_graph.freeze_graph)

In [ ]:
from src.tf import utils
importlib.reload(src.tf.utils)

In [ ]:
help(utils)

In [ ]:
utils.devices()

In [ ]:
utils.get_available_gpus()

In [ ]:
utils.is_gpu_available()


In [ ]:
utils.gpu_device_name()
